In [1]:
import pycuda.driver as drv
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [2]:
import numpy as np

In [3]:
n = 100

X = np.random.rand(n)
Y = np.random.rand(n)
Ed = np.random.rand(n)
EventID = np.sort(np.random.randint(0, 3, size=n)) 

In [4]:
import uproot
import utils
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
filenames = glob.glob('../cache/*.root')
def decode_energy(fn):
    return utils.hex_to_float(fn.split('_')[-1].replace('.root', '').replace('o', '.'))
fn = filenames[0]
energy = decode_energy(fn)

In [5]:
with uproot.open(fn) as file:
    rawEventID = file['LXe;1']['Event'].array(library='np')
    rawEd = -file['LXe;1']['Ed'].array(library='np')
    rawX = file['LXe;1']['X'].array(library='np')
    rawY = file['LXe;1']['Y'].array(library='np')
    rawZ = file['LXe;1']['Z'].array(library='np')

In [4]:
mask = rawEd > 0
mask &= (rawEventID >= 0) & (rawEventID < 10)

Ed = rawEd[mask]
X = rawX[mask]
Y = rawY[mask]
Z = rawZ[mask]
EventID = rawEventID[mask]

NameError: name 'rawEd' is not defined

In [5]:
# in
num_event = np.asarray(len(np.unique(EventID))).astype(np.int32)

# in
params = np.asarray([6., 15.0]).astype(np.float32)

In [6]:
# in
X = np.asarray(X).astype(np.float32)
Y = np.asarray(Y).astype(np.float32)
Ed = np.asarray(Ed).astype(np.float32)

# in
event_end = np.argwhere(np.diff(EventID, prepend=-1, append=num_event+1)!=0).ravel()
event_end = np.asarray(event_end).astype(np.int32)

In [7]:
# placeholders

# in
X_edge = np.zeros(2 * len(X)).astype(np.float32)
Y_edge = np.zeros(2 * len(Y)).astype(np.float32)

# in
X_edge_ID = np.zeros(2 * len(X)).astype(np.int32)
Y_edge_ID = np.zeros(2 * len(Y)).astype(np.int32)

# in
_, num_hits = np.unique(EventID, return_counts=True)
weight = np.zeros(np.sum((2*num_hits-1)**2)).astype(np.float32)

# in
recomb = np.zeros(num_event).astype(np.float32)

# in
fcache = np.zeros(2 * len(X)).astype(np.float32)
icache = np.zeros(2 * len(X)).astype(np.int32)

In [8]:
weight.nbytes/1e6

0.054188

In [9]:
gpu_kernels = open('./CudaKernels.cpp', 'r').read()
mod = SourceModule(gpu_kernels)
func = mod.get_function('calculate_recomb')

In [10]:
func(
    drv.In(X),
    drv.In(Y),
    drv.In(Ed),
    drv.In(event_end),
    drv.In(num_event),
    drv.In(params),
    drv.In(X_edge),
    drv.In(Y_edge),
    drv.In(X_edge_ID),
    drv.In(Y_edge_ID),
    drv.InOut(weight),
    drv.InOut(recomb),
    drv.In(fcache),
    drv.In(icache),
    block=(100,1,1)
)

In [11]:
len(weight)

13547

In [130]:
event_end

array([  0,  39,  72, 100], dtype=int32)

In [131]:
recomb

array([19.381504, 15.191582, 10.842622], dtype=float32)

In [132]:
np.sum(Ed[72:100])

10.842625

In [105]:
(2*39-1)**2

5929

<ipython-input-70-bbf4e80f48ee>:1: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(106): warning: variable "length" was declared but never referenced

kernel.cu(109): warning: variable "beta" was declared but never referenced

kernel.cu(106): warning: variable "length" was declared but never referenced

kernel.cu(109): warning: variable "beta" was declared but never referenced


  mod = SourceModule(gpu_kernels)


In [7]:
func = mod.get_function('test_arg_sort')
# func = mod.get_function('test_quick_sort_iarr_by_fkey')

In [14]:
_len = 100

map = np.zeros(_len).astype(np.int32)
arr = np.random.rand(_len).astype(np.float32)
length = np.asarray(_len).astype(np.int32)

In [15]:
np.argsort(arr)

array([88, 61, 76, 46, 38, 11, 35, 95,  8, 65, 63, 41, 51, 26, 89, 52, 94,
       72, 45, 28, 40, 31, 80, 14, 83, 64, 82, 15, 17, 19,  1, 21, 10, 39,
       54, 84,  2, 91,  0, 86, 30, 92, 60, 75, 78, 36,  4,  6, 16, 79, 58,
       81, 50, 59,  3,  5, 33, 34, 68, 18, 74, 49, 12, 87, 97, 53, 48, 62,
       13, 44, 27, 57, 66, 22, 70, 24, 56, 20, 90, 71, 85, 93, 73, 98,  9,
       42, 55, 99, 32, 69, 77, 25, 67, 37,  7, 29, 96, 43, 47, 23])

In [16]:
func(
    cuda.InOut(arr),
    cuda.InOut(map),
    cuda.InOut(length),
    block=(1,1,1)
)
np.all(map == np.argsort(arr))

True